In [135]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

import cfscrape

In [136]:
def crawl_microsoft_licenses():
    scraper = cfscrape.create_scraper()
    url="https://docs.microsoft.com/en-us/azure/active-directory/enterprise-users/licensing-service-plan-reference"
    scraped_html=scraper.get(url).content
    license_table = pd.read_html(scraped_html)[0]
    return license_table

In [157]:
df = crawl_microsoft_licenses()

In [158]:
from datetime import date
print("Date:",date.today())

print(len(df), "distinct licenses in total!")


Date: 2023-07-10
416 distinct licenses in total!


Converting service plans to lower

In [159]:
df['Service plans included (friendly names)']=df['Service plans included (friendly names)'].str.lower()

Extracting the list of 'Plans'

In [160]:
pattern = r"\s\([a-f0-9-]+\)"
df['Plans']= df['Service plans included (friendly names)'].apply(lambda x: re.sub(pattern, ",", x)[:-1]).str.split(',')

In [161]:
df.head()

,Product name,String ID,GUID,Service plans included,Service plans included (friendly names),Plans
0,Advanced Communications,ADV_COMMS,e4654015-5daf-4a48-9b37-4f309dddd88b,TEAMS_ADVCOMMS (604ec28a-ae18-4bc6-91b0-11da94504ba9),microsoft 365 advanced communications (604ec28a-ae18-4bc6-91b0-11da94504ba9),[microsoft 365 advanced communications]
1,AI Builder Capacity add-on,CDSAICAPACITY,d2dea78b-507c-4e56-b400-39447f4738f8,CDSAICAPACITY (a7c70a41-5e02-4271-93e6-d9b4184d83f5)EXCHANGE_S_FOUNDATION (113feb6c-3fe4-4440-bddc-54d774bf0318),ai builder capacity add-on (a7c70a41-5e02-4271-93e6-d9b4184d83f5)exchange foundation (113feb6c-3fe4-4440-bddc-54d774bf0318),"[ai builder capacity add-on, exchange foundation]"
2,App Connect IW,SPZA_IW,8f0c5670-4e56-4892-b06d-91c085d7004f,SPZA (0bfc98ed-1dbc-4a97-b246-701754e48b17)EXCHANGE_S_FOUNDATION (113feb6c-3fe4-4440-bddc-54d774bf0318),app connect (0bfc98ed-1dbc-4a97-b246-701754e48b17)exchange foundation (113feb6c-3fe4-4440-bddc-54d774bf0318),"[app connect, exchange foundation]"
3,App governance add-on to Microsoft Defender for Cloud Apps,Microsoft_Cloud_App_Security_App_Governance_Add_On,9706eed9-966f-4f1b-94f6-bb2b4af99a5b,M365_AUDIT_PLATFORM (f6de4823-28fa-440b-b886-4783fa86ddba)MICROSOFT_APPLICATION_PROTECTION_AND_GOVERNANCE_A (5f3b1ded-75c0-4b31-8e6e-9b077eaadfd5)MICROSOFT_APPLICATION_PROTECTION_AND_GOVERNANCE_D (2e6ffd72-52d1-4541-8f6c-938f9a8d4cdc),microsoft 365 audit platform (f6de4823-28fa-440b-b886-4783fa86ddba)microsoft application protection and governance (a) (5f3b1ded-75c0-4b31-8e6e-9b077eaadfd5)microsoft application protection and governance (d) (2e6ffd72-52d1-4541-8f6c-938f9a8d4cdc),"[microsoft 365 audit platform, microsoft application protection and governance, , microsoft application protection and governance, ]"
4,Microsoft 365 Audio Conferencing,MCOMEETADV,0c266dff-15dd-4b49-8397-2bb16070ed52,MCOMEETADV (3e26ee1f-8a5f-4d52-aee2-b81ce45c8f40),microsoft 365 audio conferencing (3e26ee1f-8a5f-4d52-aee2-b81ce45c8f40),[microsoft 365 audio conferencing]


## Comparing Licenses

In [183]:
searchfor = ['Azure Active Directory']

In [184]:
df_comparison = df[df['Product name'].str.contains(('|'.join(searchfor)))][['Product name', 'Plans']] 

# Collect unique plans
unique_plans = set()
for plans in df_comparison['Plans']:
    unique_plans.update(plans)

# Create columns for unique plans
for plan in unique_plans:
    df_comparison[plan] = df_comparison.apply(lambda row: plan.lower() in [p.lower() for p in row['Plans']], axis=1)

# Drop the 'Plans' column
df_comparison = df_comparison.drop('Plans', axis=1)

# Sorting by Column name
df_comparison = df_comparison.reindex(sorted(df_comparison.columns), axis=1)

# Coloring output
styled_df = df_comparison.style.applymap(lambda x: 'background-color: red' if not x else '')
styled_df


,Product name,azure active directory premium p1,azure active directory premium p2,cloud app security discovery,exchange foundation,microsoft azure active directory basic,microsoft azure multi-factor authentication,microsoft defender for cloud apps discovery
5,Azure Active Directory Basic,0,0,0,0,1,0,0
6,Azure Active Directory Premium P1,1,0,1,1,0,1,0
7,Azure Active Directory Premium P1 for faculty,1,0,0,1,0,1,1
8,Azure Active Directory Premium P2,1,1,1,1,0,1,0
